# Imports

In [53]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

In [54]:
if "notebooks" in os.getcwd():
    %cd ..

fn = Path("data/tyndp_2024_bundle/Offshore hubs/GENERATOR.xlsx")

# Read potentials

In [55]:
df_existing = pd.read_excel(fn, sheet_name="EXISTING").set_index(
    ["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO", "TECHNOLOGY"]
)[["MW"]]
df_existing

MW
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY         
BEIOH01       Hub                2030 Distributed Energy DC_FB_OH    3000.00
                                      Global Ambition    DC_FB_OH    3000.00
                                      National Trends    DC_FB_OH    3000.00
                                 2035 Distributed Energy DC_FB_OH    3000.00
                                      Global Ambition    DC_FB_OH    3000.00
...                                                                      ...
DEOR001       Radial             2045 National Trends    AC_FB_R      111.00
                                                         DC_FB_R     3549.75
                                 2050 Distributed Energy DC_FB_R     2699.75
                                      Global Ambition    DC_FB_R     2699.75
                                      National Trends    DC_FB_R     2699.75

[1371 rows x 1 columns]

In [56]:
df_layer_pot = pd.read_excel(fn, sheet_name="LAYER_POTENTIAL").set_index(
    ["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO", "TECH1", "TECH2"]
)[["EXISTING_MW", "MARGIN_MW", "MAX_MW"]]
df_layer_pot

EXISTING_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1   TECH2                
ALOR001       Radial             2030 Distributed Energy AC_FB_R NaN            0.0   
                                 2035 Distributed Energy AC_FB_R NaN            0.0   
                                 2040 Distributed Energy AC_FB_R NaN            0.0   
                                 2045 Distributed Energy AC_FB_R NaN            0.0   
                                 2050 Distributed Energy AC_FB_R NaN            0.0   
...                                                                             ...   
DEOR001       Radial             2030 National Trends    DC_FL_R NaN            0.0   
                                 2035 National Trends    DC_FL_R NaN            0.0   
                                 2040 National Trends    DC_FL_R NaN            0.0   
                                 2045 National Trends    DC_FL_R NaN            0.0   
                                 2050 National Trends    DC_FL_R NaN            0.0   

                                                                        MARGIN_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1   TECH2              
ALOR001       Radial             2030 Distributed Energy AC_FB_R NaN          0.0   
                                 2035 Distributed Energy AC_FB_R NaN          0.0   
                                 2040 Distributed Energy AC_FB_R NaN          0.0   
                                 2045 Distributed Energy AC_FB_R NaN          0.0   
                                 2050 Distributed Energy AC_FB_R NaN          0.0   
...                                                                           ...   
DEOR001       Radial             2030 National Trends    DC_FL_R NaN          0.0   
                                 2035 National Trends    DC_FL_R NaN          0.0   
                                 2040 National Trends    DC_FL_R NaN          0.0   
                                 2045 National Trends    DC_FL_R NaN          0.0   
                                 2050 National Trends    DC_FL_R NaN          0.0   

                                                                        MAX_MW  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1   TECH2          
ALOR001       Radial             2030 Distributed Energy AC_FB_R NaN       0.0  
                                 2035 Distributed Energy AC_FB_R NaN       0.0  
                                 2040 Distributed Energy AC_FB_R NaN       0.0  
                                 2045 Distributed Energy AC_FB_R NaN       0.0  
                                 2050 Distributed Energy AC_FB_R NaN       0.0  
...                                                                        ...  
DEOR001       Radial             2030 National Trends    DC_FL_R NaN       0.0  
                                 2035 National Trends    DC_FL_R NaN       0.0  
                                 2040 National Trends    DC_FL_R NaN       0.0  
                                 2045 National Trends    DC_FL_R NaN       0.0  
                                 2050 National Trends    DC_FL_R NaN       0.0  

[3180 rows x 3 columns]

In [57]:
df_zone_pot = pd.read_excel(fn, sheet_name="ZONE_POTENTIAL").set_index(
    ["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO"]
)[["EXISTING_MW", "MARGIN_MW", "MAX_MW"]]
df_zone_pot

EXISTING_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy         0.00   
                                 2035 Distributed Energy         0.00   
                                 2040 Distributed Energy         0.00   
                                 2045 Distributed Energy         0.00   
                                 2050 Distributed Energy         0.00   
...                                                               ...   
DEOR001       Radial             2030 National Trends         7548.75   
                                 2035 National Trends         7548.75   
                                 2040 National Trends         7486.75   
                                 2045 National Trends         3660.75   
                                 2050 National Trends         2699.75   

                                                          MARGIN_MW   MAX_MW  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                                
ALOR001       Radial             2030 Distributed Energy        0.0     0.00  
                                 2035 Distributed Energy        0.0     0.00  
                                 2040 Distributed Energy        0.0     0.00  
                                 2045 Distributed Energy        0.0     0.00  
                                 2050 Distributed Energy        0.0     0.00  
...                                                             ...      ...  
DEOR001       Radial             2030 National Trends           0.0  7548.75  
                                 2035 National Trends           0.0  7548.75  
                                 2040 National Trends          62.0  7548.75  
                                 2045 National Trends        3888.0  7548.75  
                                 2050 National Trends        4849.0  7548.75  

[840 rows x 3 columns]

# Compare LAYER_POTENTIAL and ZONE_POTENTIAL

In [58]:
tol = 1e-3
df = (
    df_layer_pot.groupby(by=["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO"])
    .sum()
    .join(df_zone_pot, how="outer", lsuffix="_L", rsuffix="_Z")
    .assign(
        EXISTING_MW_diff=lambda x: x.EXISTING_MW_L - x.EXISTING_MW_Z,
        MARGIN_MW_diff=lambda x: x.MARGIN_MW_L - x.MARGIN_MW_Z,
        MAX_MW_diff=lambda x: x.MAX_MW_L - x.MAX_MW_Z,
    )
)
df

EXISTING_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
ALOR001       Radial             2030 Distributed Energy         0.0000   
                                      Global Ambition            0.0000   
                                      National Trends            0.0000   
                                 2035 Distributed Energy         0.0000   
                                      Global Ambition            0.0000   
...                                                                 ...   
UKOR001       Radial             2045 Global Ambition        45240.6181   
                                      National Trends        45240.6181   
                                 2050 Distributed Energy     45990.6181   
                                      Global Ambition        45990.6181   
                                      National Trends        45990.6181   

                                                          MARGIN_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy      0.00000   
                                      Global Ambition         0.00000   
                                      National Trends         0.00000   
                                 2035 Distributed Energy      0.00000   
                                      Global Ambition         0.00000   
...                                                               ...   
UKOR001       Radial             2045 Global Ambition      2172.46668   
                                      National Trends      2172.46668   
                                 2050 Distributed Energy   1422.46668   
                                      Global Ambition      1422.46668   
                                      National Trends      1422.46668   

                                                             MAX_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy      0.00000   
                                      Global Ambition         0.00000   
                                      National Trends         0.00000   
                                 2035 Distributed Energy      0.00000   
                                      Global Ambition         0.00000   
...                                                               ...   
UKOR001       Radial             2045 Global Ambition     47413.08478   
                                      National Trends     47413.08478   
                                 2050 Distributed Energy  47413.08478   
                                      Global Ambition     47413.08478   
                                      National Trends     47413.08478   

                                                          EXISTING_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
ALOR001       Radial             2030 Distributed Energy         0.0000   
                                      Global Ambition            0.0000   
                                      National Trends            0.0000   
                                 2035 Distributed Energy         0.0000   
                                      Global Ambition            0.0000   
...                                                                 ...   
UKOR001       Radial             2045 Global Ambition        45240.6181   
                                      National Trends        45240.6181   
                                 2050 Distributed Energy     45990.6181   
                                      Global Ambition        45990.6181   
                                      National Trends        45990.6181   

                                                          MARGIN_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy    

## Check that we have the same set of existing capacities

In [59]:
df_nan = (
    df[df[["EXISTING_MW_diff", "MARGIN_MW_diff", "MAX_MW_diff"]].isna()]
    .replace(0, np.nan)
    .dropna(how="all", axis=0)
)
df_nan

,,,,EXISTING_MW_L,MARGIN_MW_L,MAX_MW_L,EXISTING_MW_Z,MARGIN_MW_Z,MAX_MW_Z,EXISTING_MW_diff,MARGIN_MW_diff,MAX_MW_diff
OFFSHORE_NODE,OFFSHORE_NODE_TYPE,YEAR,SCENARIO,,,,,,,,,


## Compare EXISTING_MW

In [60]:
diffs = [c for c in df.columns if "EXISTING" in c]
df_ex = (
    df[diffs]
    .sort_values(by="EXISTING_MW_diff", ascending=False)
    .query("abs(EXISTING_MW_diff) > @tol")
)
df_ex.head(10)

EXISTING_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
DEOH002       Hub                2045 Distributed Energy        5828.55   
                                      Global Ambition           5828.55   
                                      National Trends           5828.55   

                                                          EXISTING_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
DEOH002       Hub                2045 Distributed Energy        6354.55   
                                      Global Ambition           6354.55   
                                      National Trends           6354.55   

                                                          EXISTING_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                              
DEOH002       Hub                2045 Distributed Energy            -526.0  
                                      Global Ambition               -526.0  
                                      National Trends               -526.0

In [61]:
df_ex.tail(10)

EXISTING_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
DEOH002       Hub                2045 Distributed Energy        5828.55   
                                      Global Ambition           5828.55   
                                      National Trends           5828.55   

                                                          EXISTING_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
DEOH002       Hub                2045 Distributed Energy        6354.55   
                                      Global Ambition           6354.55   
                                      National Trends           6354.55   

                                                          EXISTING_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                              
DEOH002       Hub                2045 Distributed Energy            -526.0  
                                      Global Ambition               -526.0  
                                      National Trends               -526.0

The two tables indicates that the **existing potential** in `DEOH002`varies between the `LAYER_POTENTIAL` and `ZONE_POTENTIAL` sheets for 2045 across all scenarios. The `ZONE_POTENTIAL` sheet presents a higher value, with `LAYER_POTENTIAL` missing 526 MW in `DEOH002` in 2045.

## Compare MAX_MW

In [62]:
diffs = [c for c in df.columns if "MAX" in c]
df_max = (
    df[diffs]
    .sort_values(by="MAX_MW_diff", ascending=False)
    .query("abs(MAX_MW_diff) > @tol")
)
df_max.head(10)

MAX_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                           
FIOH001       Hub                2030 Distributed Energy  65000.000000   
                                      Global Ambition     65000.000000   
                                      National Trends     65000.000000   
DEOH001       FarShoreHub        2030 Distributed Energy  62868.000000   
FIOH001       Hub                2035 Distributed Energy  65000.000000   
DEOH001       FarShoreHub        2030 Global Ambition     62868.000000   
                                      National Trends     62868.000000   
FIOH001       Hub                2035 Global Ambition     65000.000000   
                                      National Trends     65000.000000   
FROH002       Hub                2030 Global Ambition     38809.913867   

                                                              MAX_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                           
FIOH001       Hub                2030 Distributed Energy   7000.000000   
                                      Global Ambition      7000.000000   
                                      National Trends      7000.000000   
DEOH001       FarShoreHub        2030 Distributed Energy  18868.000000   
FIOH001       Hub                2035 Distributed Energy  21000.000000   
DEOH001       FarShoreHub        2030 Global Ambition     18868.000000   
                                      National Trends     18868.000000   
FIOH001       Hub                2035 Global Ambition     21000.000000   
                                      National Trends     21000.000000   
FROH002       Hub                2030 Global Ambition      2028.742923   

                                                           MAX_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
FIOH001       Hub                2030 Distributed Energy  58000.000000  
                                      Global Ambition     58000.000000  
                                      National Trends     58000.000000  
DEOH001       FarShoreHub        2030 Distributed Energy  44000.000000  
FIOH001       Hub                2035 Distributed Energy  44000.000000  
DEOH001       FarShoreHub        2030 Global Ambition     44000.000000  
                                      National Trends     44000.000000  
FIOH001       Hub                2035 Global Ambition     44000.000000  
                                      National Trends     44000.000000  
FROH002       Hub                2030 Global Ambition     36781.170944

In [63]:
df_max.tail(10)

MAX_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                           
NOSOR01       Radial             2045 Distributed Energy   2046.442403   
UKOH003       Hub                2045 National Trends     16914.263202   
                                      Distributed Energy  16914.263202   
                                      Global Ambition     16914.263202   
DEOH002       Hub                2045 Global Ambition      5828.550000   
                                 2050 National Trends      5828.550000   
                                      Global Ambition      5828.550000   
                                      Distributed Energy   5828.550000   
                                 2045 National Trends      5828.550000   
                                      Distributed Energy   5828.550000   

                                                              MAX_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                           
NOSOR01       Radial             2045 Distributed Energy   1909.712163   
UKOH003       Hub                2045 National Trends     16827.086690   
                                      Distributed Energy  16827.086690   
                                      Global Ambition     16827.086690   
DEOH002       Hub                2045 Global Ambition      6354.550000   
                                 2050 National Trends      6354.550000   
                                      Global Ambition      6354.550000   
                                      Distributed Energy   6354.550000   
                                 2045 National Trends      6354.550000   
                                      Distributed Energy   6354.550000   

                                                          MAX_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                         
NOSOR01       Radial             2045 Distributed Energy   136.730240  
UKOH003       Hub                2045 National Trends       87.176512  
                                      Distributed Energy    87.176512  
                                      Global Ambition       87.176512  
DEOH002       Hub                2045 Global Ambition     -526.000000  
                                 2050 National Trends     -526.000000  
                                      Global Ambition     -526.000000  
                                      Distributed Energy  -526.000000  
                                 2045 National Trends     -526.000000  
                                      Distributed Energy  -526.000000

As can be seen from the two tables above, the `LAYER_POTENTIAL` and `ZONE_POTENTIAL` sheets in `Offshore hubs/GENERATOR.xlsx` show discrepancies in the **maximum potential** specified per node, year, and scenario.

The existing 526 MW deficit in `DEOH002` in `LAYER_POTENTIAL` accounts for the observed negative difference of 526 MW.

The 58 GW difference in `FIOH001` can be understood by analysing all the planning horizons, as the `MAX_MW` of `ZONE_POTENTIAL` changes over time.

# Compare EXISTING and LAYER_POTENTIAL

In [64]:
df = (
    df_existing.replace("H2_", "DC_", regex=True)
    .groupby(
        by=["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO", "TECHNOLOGY"]
    )
    .sum()
    .join(
        (
            df_layer_pot.rename_axis(index={"TECH1": "TECHNOLOGY"})
            .droplevel("TECH2")
            .rename(columns={"EXISTING_MW": "MW"})[["MW"]]
        ),
        how="outer",
        rsuffix="2",
    )
    .assign(MW_diff=lambda x: x.MW - x.MW2)
)
df

MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY                 
ALOR001       Radial             2030 Distributed Energy AC_FB_R              NaN   
                                                         AC_FL_R              NaN   
                                                         DC_FB_R              NaN   
                                                         DC_FL_R              NaN   
                                      Global Ambition    AC_FB_R              NaN   
...                                                                           ...   
UKOR001       Radial             2050 Global Ambition    DC_FL_R     14962.282264   
                                      National Trends    AC_FB_R      8975.976664   
                                                         AC_FL_R        28.863405   
                                                         DC_FB_R     22023.495767   
                                                         DC_FL_R     14962.282264   

                                                                              MW2  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY                 
ALOR001       Radial             2030 Distributed Energy AC_FB_R         0.000000   
                                                         AC_FL_R         0.000000   
                                                         DC_FB_R         0.000000   
                                                         DC_FL_R         0.000000   
                                      Global Ambition    AC_FB_R         0.000000   
...                                                                           ...   
UKOR001       Radial             2050 Global Ambition    DC_FL_R     14962.282264   
                                      National Trends    AC_FB_R      8975.976664   
                                                         AC_FL_R        28.863405   
                                                         DC_FB_R     22023.495767   
                                                         DC_FL_R     14962.282264   

                                                                     MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY           
ALOR001       Radial             2030 Distributed Energy AC_FB_R         NaN  
                                                         AC_FL_R         NaN  
                                                         DC_FB_R         NaN  
                                                         DC_FL_R         NaN  
                                      Global Ambition    AC_FB_R         NaN  
...                                                                      ...  
UKOR001       Radial             2050 Global Ambition    DC_FL_R         0.0  
                                      National Trends    AC_FB_R         0.0  
                                                         AC_FL_R         0.0  
                                                         DC_FB_R         0.0  
                                                         DC_FL_R         0.0  

[3249 rows x 3 columns]

## Check that we have the same set of existing capacities

In [65]:
df_nan = df[df["MW_diff"].isna()].replace(0, np.nan).dropna(how="all", axis=0)
df_nan

MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY                
BEOH001       Hub                2030 Distributed Energy DC_FB_OH            NaN   
                                                         DC_FB_R     2647.515945   
                                      Global Ambition    DC_FB_OH            NaN   
                                                         DC_FB_R     2647.515945   
                                      National Trends    DC_FB_OH            NaN   
...                                                                          ...   
SEOH002       Hub                2050 Distributed Energy DC_FB_R     1399.438094   
                                      Global Ambition    DC_FB_OH            NaN   
                                                         DC_FB_R     1399.438094   
                                      National Trends    DC_FB_OH            NaN   
                                                         DC_FB_R     1399.438094   

                                                                             MW2  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY                
BEOH001       Hub                2030 Distributed Energy DC_FB_OH    2647.515945   
                                                         DC_FB_R             NaN   
                                      Global Ambition    DC_FB_OH    2647.515945   
                                                         DC_FB_R             NaN   
                                      National Trends    DC_FB_OH    2647.515945   
...                                                                          ...   
SEOH002       Hub                2050 Distributed Energy DC_FB_R             NaN   
                                      Global Ambition    DC_FB_OH    1399.438094   
                                                         DC_FB_R             NaN   
                                      National Trends    DC_FB_OH    1399.438094   
                                                         DC_FB_R             NaN   

                                                                     MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY           
BEOH001       Hub                2030 Distributed Energy DC_FB_OH        NaN  
                                                         DC_FB_R         NaN  
                                      Global Ambition    DC_FB_OH        NaN  
                                                         DC_FB_R         NaN  
                                      National Trends    DC_FB_OH        NaN  
...                                                                      ...  
SEOH002       Hub                2050 Distributed Energy DC_FB_R         NaN  
                                      Global Ambition    DC_FB_OH        NaN  
                                                         DC_FB_R         NaN  
                                      National Trends    DC_FB_OH        NaN  
                                                         DC_FB_R         NaN  

[93 rows x 3 columns]

In [66]:
(
    df_nan.reset_index(level=4)
    .replace("_R", "_OH", regex=True)  # Ignore reallocations for PEMMDB alignment
    .groupby(
        by=["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO", "TECHNOLOGY"]
    )
    .sum()
    .assign(MW_diff=lambda x: x.MW - x.MW2)
    .query("abs(MW_diff) > @tol")
    .sort_values(by="MW_diff", ascending=False)
)

MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY            
NLOH001       FarShoreHub        2050 National Trends    H2_FB_OH    20000.0   
                                      Distributed Energy H2_FB_OH    20000.0   
                                      Global Ambition    H2_FB_OH    20000.0   
                                 2045 Distributed Energy H2_FB_OH    16000.0   
                                      Global Ambition    H2_FB_OH    16000.0   
                                      National Trends    H2_FB_OH    16000.0   
                                 2040 Global Ambition    H2_FB_OH    12000.0   
                                      National Trends    H2_FB_OH    12000.0   
                                      Distributed Energy H2_FB_OH    12000.0   
DEOH001       FarShoreHub        2035 National Trends    DC_FB_OH    10868.0   
                                      Global Ambition    DC_FB_OH    10868.0   
                                      Distributed Energy DC_FB_OH    10868.0   
                                 2040 National Trends    DC_FB_OH    10468.0   
                                      Global Ambition    DC_FB_OH    10468.0   
                                      Distributed Energy DC_FB_OH    10468.0   
                                 2045 Global Ambition    DC_FB_OH     9680.0   
                                      National Trends    DC_FB_OH     9680.0   
                                      Distributed Energy DC_FB_OH     9680.0   
                                 2030 Global Ambition    DC_FB_OH     8868.0   
                                      Distributed Energy DC_FB_OH     8868.0   
                                      National Trends    DC_FB_OH     8868.0   
                                 2050 Distributed Energy DC_FB_OH     8780.0   
                                      Global Ambition    DC_FB_OH     8780.0   
                                      National Trends    DC_FB_OH     8780.0   
DKWOH01       FarShoreHub        2050 National Trends    H2_FB_OH     7500.0   
                                      Global Ambition    H2_FB_OH     7500.0   
                                      Distributed Energy H2_FB_OH     7500.0   
NLOH001       FarShoreHub        2035 National Trends    H2_FB_OH     4350.0   
                                      Global Ambition    H2_FB_OH     4350.0   
                                      Distributed Energy H2_FB_OH     4350.0   
DKWOH01       FarShoreHub        2045 Distributed Energy H2_FB_OH     3750.0   
                                      National Trends    H2_FB_OH     3750.0   
                                      Global Ambition    H2_FB_OH     3750.0   
                                 2050 National Trends    DC_FB_OH     3000.0   
                                 2045 Distributed Energy DC_FB_OH     3000.0   
                                 2050 Global Ambition    DC_FB_OH     3000.0   
                                      Distributed Energy DC_FB_OH     3000.0   
                                 2040 Distributed Energy DC_FB_OH     3000.0   
                                 2045 National Trends    DC_FB_OH     3000.0   
                                      Global Ambition    DC_FB_OH     3000.0   
                                 2040 Global Ambition    DC_FB_OH     3000.0   
                                      National Trends    DC_FB_OH     3000.0   
                                 2035 National Trends    DC_FB_OH     1000.0   
                                      Global Ambition    DC_FB_OH     1000.0   
                                      Distributed Energy DC_FB_OH     1000.0   

                                                                     MW2  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY        
NLOH001       FarShoreHub        2050 National Trends    H2_FB_OH    0.0   
                                      Distributed Energy H2_FB_OH    0.0   
         

The table indicates that the `EXISTING` sheet specifies a greater number of existing capacity locations compared to the `LAYER_POTENTIAL` sheet. Note that this table ignores reallocation for PEMMDB alignment.

## Check values for common indexes

In [67]:
df_diff = df.query("abs(MW_diff) > @tol").sort_values(by="MW_diff")[
    ["MW", "MW2", "MW_diff"]
]
df_diff

MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY            
NLOH001       FarShoreHub        2050 National Trends    DC_FB_OH    11300.0   
                                      Distributed Energy DC_FB_OH    11300.0   
                                      Global Ambition    DC_FB_OH    11300.0   
                                 2045 National Trends    DC_FB_OH     8300.0   
                                      Global Ambition    DC_FB_OH     8300.0   
                                      Distributed Energy DC_FB_OH     8300.0   
                                 2040 Distributed Energy DC_FB_OH     5300.0   
                                      Global Ambition    DC_FB_OH     5300.0   
                                      National Trends    DC_FB_OH     5300.0   
DEOH001       FarShoreHub        2035 Distributed Energy DC_FB_OH    28000.0   
                                      Global Ambition    DC_FB_OH    28000.0   
                                      National Trends    DC_FB_OH    28000.0   
DKWOH01       FarShoreHub        2050 Global Ambition    DC_FB_OH    19500.0   
                                      National Trends    DC_FB_OH    19500.0   
                                      Distributed Energy DC_FB_OH    19500.0   
DEOH001       FarShoreHub        2040 National Trends    DC_FB_OH    42000.0   
                                      Global Ambition    DC_FB_OH    42000.0   
                                      Distributed Energy DC_FB_OH    42000.0   
                                 2045 Distributed Energy DC_FB_OH    52000.0   
                                      National Trends    DC_FB_OH    52000.0   
                                      Global Ambition    DC_FB_OH    52000.0   
                                 2030 Distributed Energy DC_FB_OH    10000.0   
                                      Global Ambition    DC_FB_OH    10000.0   
                                      National Trends    DC_FB_OH    10000.0   
                                 2050 Distributed Energy DC_FB_OH    54000.0   
                                      Global Ambition    DC_FB_OH    54000.0   
                                      National Trends    DC_FB_OH    54000.0   
DKWOH01       FarShoreHub        2045 Global Ambition    DC_FB_OH    15250.0   
                                      National Trends    DC_FB_OH    15250.0   
                                      Distributed Energy DC_FB_OH    15250.0   
NLOH001       FarShoreHub        2035 Distributed Energy DC_FB_OH     5300.0   
                                      Global Ambition    DC_FB_OH     5300.0   
                                      National Trends    DC_FB_OH     5300.0   
DKWOH01       FarShoreHub        2040 Global Ambition    DC_FB_OH    11000.0   
                                      Distributed Energy DC_FB_OH    11000.0   
                                      National Trends    DC_FB_OH    11000.0   
                                 2035 National Trends    DC_FB_OH     6000.0   
                                      Distributed Energy DC_FB_OH     6000.0   
                                      Global Ambition    DC_FB_OH     6000.0   

                                                                         MW2  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY            
NLOH001       FarShoreHub        2050 National Trends    DC_FB_OH    31300.0   
                                      Distributed Energy DC_FB_OH    31300.0   
                                      Global Ambition    DC_FB_OH    31300.0   
                                 2045 National Trends    DC_FB_OH    24300.0   
                                      Global Ambition    DC_FB_OH    24300.0   
                                      Distributed Energy DC_FB_OH    24300.0   
                                 2040 Distributed Energy DC_FB_OH    17300.0   
                                      Global Ambition    DC_FB_OH    1730

However, the `ZONE_POTENTIAL` sheet defines additional capacities for the common locations.